In [ ]:
from pathlib import Path

import numpy as np
from aiida import engine, load_profile, orm
from aiida_quantum_transport.workchains import CoulombDiamondsWorkChain
from ase.io import read

load_profile()

# Comments

- The tester uses `ase` to load the example structures, so you'll need to install `ase`.

# Data source

In [ ]:
DATA_DIR = Path("pentacene/data")

# Leads input

In [ ]:
path = DATA_DIR / "leads.xyz"

atoms = read(path.as_posix())
leads_structure = orm.StructureData(ase=atoms)

leads_parameters = orm.Dict(
    {
        "h": 0.2,
        "xc": "LDA",
        "nbands": "nao",
        "convergence": {
            "bands": "all",
        },
        "basis": "szp(dzp)",
        "occupations": {
            "name": "fermi-dirac",
            "width": 0.01,
        },
        "mode": "lcao",
        "mixer": {
            "method": "separate",
            "backend": "pulay",
            "beta": 0.02,
            "nmaxold": 5,
            "weight": 100.0,
        },
        "symmetry": {
            "point_group": False,
            "time_reversal": True,
        },
    }
)

leads_kpoints = orm.KpointsData()
leads_kpoints.set_kpoints_mesh([3, 1, 1])

leads_output_prefix = orm.Str("leads")

# Device input

In [ ]:
path = DATA_DIR / "device.xyz"

atoms = read(path.as_posix())
device_structure = orm.StructureData(ase=atoms)

device_parameters = orm.Dict(
    {
        "h": 0.2,
        "xc": "LDA",
        "nbands": "nao",
        "convergence": {
            "bands": "all",
        },
        "basis": "szp(dzp)",
        "occupations": {
            "name": "fermi-dirac",
            "width": 0.01,
        },
        "mode": "lcao",
        "mixer": {
            "method": "separate",
            "backend": "pulay",
            "beta": 0.02,
            "nmaxold": 5,
            "weight": 100.0,
        },
        "parallel": {
            "band": 1,
            "augment_grids": True,
            "sl_auto": True,
        },
    }
)

device_kpoints = orm.KpointsData()
device_kpoints.set_kpoints_mesh([1, 1, 1])

device_output_prefix = orm.Str("device")

# Green's Function

In [ ]:
species = ["C", "H"]
number_of_spherical_harmonics = [9, 4]
basis = {
    species: number
    for species, number in zip(species, number_of_spherical_harmonics)
}

# Metadata

In [ ]:
h, m, s = 1, 0, 0

NODES = 1
TASKS_PER_NODE = 1
CPUS_PER_TASK = 1
OMP_NUM_THREADS = CPUS_PER_TASK
NUMBA_NUM_THREADS = CPUS_PER_TASK * TASKS_PER_NODE

metadata = {
    "options": {
        "withmpi": True,
        "resources": {
            "num_machines": NODES,
            "num_mpiprocs_per_machine": TASKS_PER_NODE,
            "num_cores_per_mpiproc": CPUS_PER_TASK,
        },
        "max_wallclock_seconds": h * 3600 + m * 60 + s,
        "environment_variables": {
            "OMP_NUM_THREADS": OMP_NUM_THREADS,
            "NUMBA_NUM_THREADS": NUMBA_NUM_THREADS,
        },
    }
}

# Workflow

In [ ]:
inputs = {
    "dft": {
        "code": orm.load_code("dft-script"),
        "leads": {
            "structure": leads_structure,
            "kpoints": leads_kpoints,
            "parameters": leads_parameters,
        },
        "device": {
            "structure": device_structure,
            "kpoints": device_kpoints,
            "parameters": device_parameters,
        },
    },
    "scattering": {
        # "region": orm.Dict(
        #     {
        #         "x_min": 16.0,
        #         "x_max": 20.0,
        #         "y_min": 4.0,
        #         "y_max": 8.0,
        #     }
        # ),
        "active": orm.Dict({"C": 3}),
    },
    "localization": {
        "code": orm.load_code("los-script"),
        "lowdin": orm.Bool(True),
    },
    "greens_function": {
        "code": orm.load_code("greens-script"),
        "basis": orm.Dict(basis),
    },
    # "greens_function_parameters": orm.Dict(
    #     {
    #         "solver": "dyson",
    #         "eta": 1e-4,
    #     }
    # ),
    # "energy_grid_parameters": orm.Dict(
    #     {
    #         "E_min": -3.0,
    #         "E_max": 3.0,
    #         "E_step": 1e-2,
    #         "matsubara_grid_scalar": 1.0,
    #     }
    # ),
    "hybridization": {
        "code": orm.load_code("hybr-script"),
        "temperature": orm.Float(300.0),
        "matsubara_grid_size": orm.Int(3000),
        "metadata": metadata,
    },
    "dmft": {
        "code": orm.load_code("dmft-script"),
        "parameters": orm.Dict(
            {
                # "U": 4.0,
                # "number_of_baths": 4,
                "tolerance": 200,
                # "alpha": 0.0,
                # "inner_max_iter": 1000,
                # "outer_max_iter": 1000,
            }
        ),
        "converge_mu": {
            "adjust_mu": orm.Bool(True),
        },
        "sweep_mu": {
            "parameters": orm.Dict(
                {
                    "dmu_min": 0.0,
                    "dmu_max": 1.0,
                    "dmu_step": 0.5,
                }
            ),
        },
    },
    "transmission": {
        "code": orm.load_code("trans-script"),
        "metadata": metadata,
    },
    "current": {
        "code": orm.load_code("curr-script"),
        # "parameters": orm.Dict({
        #     "V_min": -2.5,
        #     "V_max": 2.5,
        #     "dV": 0.1,
        # }),
    },
}

In [ ]:
node = engine.submit(CoulombDiamondsWorkChain, **inputs)


# Tests

In [ ]:
# u1, u2, u3 = node.uuid[:2], node.uuid[2:4], node.uuid[4:]
# folder = Path("~/aiida_run") / u1 / u2 / u3
# ! ls $folder
# # ! cat $folder/leads.txt


In [ ]:
# with node.outputs.nao_file.as_path() as filepath:
#     array = np.load(filepath)
# array

In [ ]:
# folder: orm.FolderData = node.outputs.dmft.converge_mu.delta_folder
# with folder.as_path() as filepath:
#     for file in filepath.iterdir():
#         data = np.load(file)